# Career Compass: AI-Powered Job Pathway Designer with GenAI

This notebook demonstrates a Generative AI application that converts resume or LinkedIn profile data into a personalized roadmap for career transition. It identifies potential new roles, skill gaps, and curated resources.

**Use Case**: Personalized career guidance for students and professionals transitioning between industries.

**GenAI Capabilities Used**:
- Document Understanding
- Few-shot Prompting
- Structured Output (JSON format)
- Grounding (Google Search)

📂 **Before You Run This Notebook:**
To use this notebook, you must upload your resume as a PDF file.

1. Click **Add Data** > **Upload** on the right-hand sidebar.
2. Select your resume file and ensure it is named `file.pdf`.
3. The uploaded file will be stored in `/kaggle/input/`.

🔄 **Alternatively**: If you prefer not to upload a resume, you can comment out the PDF cell (Cell 5) and uncomment the LinkedIn cell (Cell 6), then paste in the public URL to your LinkedIn profile. Note that many LinkedIn pages require authentication and may not always load successfully in this environment.

# Install Dependencies

In [ ]:
!pip install -q -U google-genai

# Import Libraries

In [ ]:
import os
import json
import pathlib
import httpx
from pprint import pprint
from google import genai
from google.genai import types
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# Set API Key

In [ ]:
# Code snippet from Kaggle Secrets Manager
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

GEMINI_API_KEY = user_secrets.get_secret("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise EnvironmentError("Gemini API key not found. Please set the GEMINI_API_KEY environment variable.")

client = genai.Client(api_key=GEMINI_API_KEY)

# Prompt Configuration (Few-Shot + Schema)

In [ ]:
json_schema = '''
CareerPath = {
  "job_titles": list[str],
  "skills_to_learn": list[str],
  "recommended_certifications": list[str],
  "justification": dict[str, str]
}
Return: CareerPath as JSON
'''

few_shot_examples = """
Example 1:
Resume: Former barista with experience in customer service, cash handling, and shift management.
CareerPath: {"job_titles": ["Customer Support Associate", "Sales Development Representative"], "skills_to_learn": ["CRM tools", "Salesforce basics"], "recommended_certifications": ["Google IT Support"], "justification": {"Customer Support Associate": "High alignment with communication skills", "Sales Development Representative": "Good fit due to customer engagement experience"}}

Example 2:
Resume: Biology graduate with lab research experience and data analysis skills in Excel.
CareerPath: {"job_titles": ["Data Analyst", "Research Assistant"], "skills_to_learn": ["SQL", "Python for Data Analysis"], "recommended_certifications": ["IBM Data Analyst"], "justification": {"Data Analyst": "Excel and scientific analysis background is strong foundation", "Research Assistant": "Direct fit from lab experience"}}

Example 3:
Resume: High school teacher looking to transition to instructional design or edtech.
CareerPath: {"job_titles": ["Instructional Designer", "EdTech Specialist"], "skills_to_learn": ["eLearning tools", "UI/UX for education"], "recommended_certifications": ["Coursera Learning Design"], "justification": {"Instructional Designer": "Teaching background provides natural fit", "EdTech Specialist": "Experience with curriculum development is applicable"}}
"""

# Upload and Process Local Resume PDF

In [ ]:
# === DEFAULT INPUT METHOD ===
# This cell runs by default. Ensure 'file.pdf' is uploaded to your notebook before running.
# If you prefer using a LinkedIn URL instead, comment out this cell and uncomment the next one.

from pathlib import Path

filepath = Path("/kaggle/input/...")
if not filepath.exists():
    raise FileNotFoundError(" 'file.pdf' was not found. Please upload your resume using the 'Add Data > Upload' menu before running this notebook.")

prompt = f"""
{few_shot_examples}

Now extract a CareerPath object from this resume. Use this JSON schema:
{json_schema}
"""

pdf_response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[
        types.Part.from_bytes(
            data=filepath.read_bytes(),
            mime_type="application/pdf",
        ),
        prompt
    ]
)

# LinkedIn Page Input (Alternative to Resume)

In [ ]:
# === OPTIONAL INPUT METHOD ===
# Uncomment this section to use your LinkedIn profile instead of your resume.

# linkedin_url = "https://www.linkedin.com/in/sampleprofile"
# try:
#     linkedin_data = httpx.get(linkedin_url).content
# except Exception as e:
#     raise RuntimeError(f"Failed to retrieve LinkedIn URL: {e}")

# prompt = f"""
# {few_shot_examples}
# 
# Now extract a CareerPath object from the following LinkedIn profile HTML.
# Use this JSON schema:
# {json_schema}
# """

# linkedin_response = client.models.generate_content(
#     model="gemini-2.0-flash",
#     contents=[
#         types.Part.from_bytes(
#             data=linkedin_data,
#             mime_type="text/html",
#         ),
#         prompt
#     ]
# )

# Convert Response to JSON

In [ ]:
try:
    # Extract the JSON from the response text
    response_text = pdf_response.text or linkedin_response.text
    
    # Look for JSON content within markdown code blocks
    if "```json" in response_text:
        # Extract content between ```json and ```
        json_text = response_text.split("```json")[1].split("```")[0].strip()
    else:
        # If no markdown formatting, use the raw text
        json_text = response_text
    
    # Now parse the extracted JSON content
    career_plan = json.loads(json_text)
    
    # Print the parsed career plan to verify
    print("Successfully parsed career plan:")
    pprint(career_plan)
    
except Exception as e:
    raise ValueError(f"Failed to parse AI response as JSON: {e}")

# Grounding with Google Search

In [ ]:
# Use Google's built-in search tool with grounding metadata
model_id = "gemini-2.0-flash"
google_search_tool = Tool(google_search=GoogleSearch())

if "skills_to_learn" not in career_plan or not career_plan["skills_to_learn"]:
    raise ValueError("No skills found in career plan. Unable to proceed with grounded search.")

grounding_examples = """Here are examples of how to find online courses for new skills:

Skill: SQL
Response:
- Codecademy SQL Track: Beginner-friendly SQL tutorials with hands-on labs.
- Google Data Analytics Certificate (Coursera): Introduces SQL in real-world business settings.

Skill: Salesforce basics
Response:
- Salesforce Trailhead: Official learning path for Salesforce beginners.
- Udemy Salesforce Admin Crash Course: Step-by-step instruction for fast learners.

Skill: UI/UX for Education
Response:
- Coursera UI/UX Design Specialization: Beginner to advanced UX design for real applications.
- edX Human-Centered Design in EdTech: Focus on educational user experience.
"""

search_prompt = f"""{grounding_examples}

Now find 2–3 reputable online courses or certifications for the following skills:
{', '.join(career_plan['skills_to_learn'])}
Return results in the same format as above.
"""

search_response = client.models.generate_content(
    model=model_id,
    contents=search_prompt,
    config=GenerateContentConfig(
        tools=[google_search_tool],
        response_modalities=["TEXT"]
    )
)

# Display Roadmap + Grounded Search Results

In [ ]:
print("--- Career Roadmap ---\n")
pprint(career_plan)

print("\n\n--- Recommended Learning Resources (via Google Search Grounding) ---\n")
for each in search_response.candidates[0].content.parts:
    print(each.text)

# Conclusion & Reflections

In this notebook, we explored how Generative AI can:
- Analyze semi-structured resume content (PDF or LinkedIn profile)
- Generate personalized career roadmap outputs in structured JSON
- Recommend job roles and explain fit
- Suggest skill development areas
- Use Google's built-in search grounding to suggest up-to-date learning resources

...all while using few-shot prompting.

This application can support students and job seekers in navigating career pivots with clarity and confidence. Future expansions could include resume editing, interview prep, or integration with job boards and online learning platforms.